In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!git clone https://github.com/omerAJ/VLM-Finetuning-PSL
!ls

Cloning into 'VLM-Finetuning-PSL'...
remote: Enumerating objects: 1206, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 1206 (delta 155), reused 131 (delta 70), pack-reused 983 (from 2)
Receiving objects: 100% (1206/1206), 59.29 MiB | 68.21 MiB/s, done.
Resolving deltas: 100% (514/514), done.
drive  sample_data  VLM-Finetuning-PSL


In [ ]:
%cd VLM-Finetuning-PSL
!git pull

[Errno 2] No such file or directory: 'VLM-Finetuning-PSL'
/content/VLM-Finetuning-PSL/internvl/internvl_chat
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 7 (delta 6), reused 7 (delta 6), pack-reused 0 (from 0)
Unpacking objects: 100% (7/7), 625 bytes | 312.00 KiB/s, done.
From https://github.com/omerAJ/VLM-Finetuning-PSL
   57d877a..13e8f2f  main       -> origin/main
Updating 57d877a..13e8f2f
Fast-forward
 internvl/internvl_chat_llava/llava/train/train.py | 1 +
 1 file changed, 1 insertion(+)


In [ ]:
# Installing reqs
%cd VLM-Finetuning-PSL/internvl
!pip install -r requirements.txt

/content/VLM-Finetuning-PSL/internvl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 12.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 96.7 

-----------
pretrained model

In [ ]:
# !hf download OpenGVLab/InternVL3-1B --local-dir /content/drive/MyDrive/finetuning-output

In [ ]:
%cd /content/VLM-Finetuning-PSL/internvl
!mkdir -p pretrained
%cd pretrained
!hf download OpenGVLab/InternVL3-1B --local-dir InternVL3-1B

/content/VLM-Finetuning-PSL/internvl
/content/VLM-Finetuning-PSL/internvl/pretrained
Fetching 20 files:   0% 0/20 [00:00<?, ?it/s]Downloading 'configuration_internvl_chat.py' to 'InternVL3-1B/.cache/huggingface/download/DsLREbI1OaCt0aQRMkT6-8j8O0Q=.502772cc6e24e6d1fbc5bf91ce76c9b234101eaf.incomplete'

configuration_internvl_chat.py: 4.04kB [00:00, 23.3MB/s]
Download complete. Moving file to InternVL3-1B/configuration_internvl_chat.py

config.json: 0.00B [00:00, ?B/s]

config.json: 6.33kB [00:00, 14.1MB/s]
Download complete. Moving file to InternVL3-1B/config.json
configuration_intern_vit.py: 5.55kB [00:00, 12.8MB/s]
Download complete. Moving file to InternVL3-1B/configuration_intern_vit.py

conversation.py: 15.3kB [00:00, 8.42MB/s]

added_tokens.json:   0% 0.00/790 [00:00<?, ?B/s]Download complete. Moving file to InternVL3-1B/conversation.py
added_tokens.json: 100% 790/790 [00:00<00:00, 7.65MB/s]
Download complete. Moving file to InternVL3-1B/added_tokens.json

image1.jpg:   0% 0.00/78

-------------------
Structuring dataset into jsonl

In [ ]:
import os
import json

video_dir = "/content/drive/MyDrive/PSL_Data_segmentation/Words"
jsonl_path = "/content/VLM-Finetuning-PSL/internvl/internvl_chat/shell/data/dataset.jsonl"

with open(jsonl_path, "w") as out_file:
    for idx, filename in enumerate(os.listdir(video_dir)):
        if filename.endswith(".mp4"):
            word = os.path.splitext(filename)[0]  # title is the label
            entry = {
                "type": "video",
                "url": filename,
                "conversations": [
                    {"from": "human", "value": "<video>\nWhich word does the person sign in this video?"},
                    {"from": "gpt", "value": word}
                ]
            }
            out_file.write(json.dumps(entry) + "\n")




In [ ]:
# create data json
import json
%cd /content/VLM-Finetuning-PSL
data = {
    "train": {
        "root": "/content/drive/MyDrive/PSL_Data_segmentation/Words",
        "annotation": "/content/VLM-Finetuning-PSL/internvl/internvl_chat/shell/data/dataset.jsonl",
        "data_augment": False,
        "max_dynamic_patch": 12,
        "repeat_time": 1,
        "length": 51
    }
}

with open("config.json", "w") as f:
    json.dump(data, f, indent=2)

/content/VLM-Finetuning-PSL


In [ ]:
!pip install transformers==4.37.2

  Using cached transformers-4.37.2-py3-none-any.whl.metadata (129 kB)
  Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.37.2-py3-none-any.whl (8.4 MB)
Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.54.1
    Uninstalling transformers-4.54.1:
      Successfully uninstalled transformers-4.54.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.37.2 which is incompatible.


In [ ]:
# Using 8 GPUs, fine-tune the full LLM, cost about 30G per GPU
%cd /content/VLM-Finetuning-PSL/internvl/internvl_chat
!GPUS=1 PER_DEVICE_BATCH_SIZE=32 sh /content/VLM-Finetuning-PSL/internvl/internvl_chat/shell/internvl3.0/2nd_finetune/internvl3_1b_dynamic_res_2nd_finetune_full.sh

/content/VLM-Finetuning-PSL/internvl/internvl_chat
+ GPUS=1
+ BATCH_SIZE=128
+ PER_DEVICE_BATCH_SIZE=32
+ GRADIENT_ACC=4
+ pwd
+ export PYTHONPATH=/env/python:/content/VLM-Finetuning-PSL/internvl/internvl_chat
+ export MASTER_PORT=34229
+ export TF_CPP_MIN_LOG_LEVEL=3
+ export LAUNCHER=pytorch
+ OUTPUT_DIR=/content/drive/MyDrive/finetuning-output-Take1
+ [ ! -d /content/drive/MyDrive/finetuning-output-Take1 ]
+ torchrun --nnodes=1 --node_rank=0 --master_addr=127.0.0.1 --nproc_per_node=1 --master_port=34229 internvl/train/internvl_chat_finetune.py --model_name_or_path OpenGVLab/InternVL3-1B --conv_style internvl2_5 --use_fast_tokenizer False --output_dir /content/drive/MyDrive/finetuning-output-Take1 --meta_path /content/VLM-Finetuning-PSL/config.json --overwrite_output_dir True --force_image_size 448 --max_dynamic_patch 12 --down_sample_ratio 0.5 --drop_path_rate 0.0 --freeze_llm False --freeze_mlp False --freeze_backbone False --vision_select_layer -1 --dataloader_num_workers 4 --bf16

Evaluating

In [ ]:
!pip show transformers

Name: transformers
Version: 4.37.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [ ]:
!pip install av
!pip install --upgrade transformers

  Using cached transformers-4.54.1-py3-none-any.whl.metadata (41 kB)
  Using cached tokenizers-0.21.4-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.54.1-py3-none-any.whl (11.2 MB)
Using cached tokenizers-0.21.4-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2


In [ ]:
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig
import torch

model_checkpoint = "OpenGVLab/InternVL3-8B-hf"
quantization_config = BitsAndBytesConfig(load_in_4bit=False)
processor = AutoProcessor.from_pretrained(model_checkpoint)
model = AutoModelForImageTextToText.from_pretrained(model_checkpoint, quantization_config=quantization_config)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "url": "/content/drive/MyDrive/PSL_Data_segmentation/Words/you.mp4",
            },
            {"type": "text", "text": "Which word does the person sign in this video?"},
        ],
    }
]
inputs = processor.apply_chat_template(
    messages,
    return_tensors="pt",
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    num_frames=8,
).to(model.device, dtype=torch.float16)

output = model.generate(**inputs, max_new_tokens=25)

decoded_output = processor.decode(output[0, inputs["input_ids"].shape[1] :], skip_special_tokens=True)
decoded_output

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

'The word that the person signs in this video is "please".'

In [ ]:
%cd /content/VLM-Finetuning-PSL/internvl/internvl_chat
%pwd

/content/VLM-Finetuning-PSL/internvl/internvl_chat


'/content/VLM-Finetuning-PSL/internvl/internvl_chat'

In [ ]:

## eval for full finetuning

from transformers import AutoProcessor, AutoModelForCausalLM
import torch
%cd /content/VLM-Finetuning-PSL/internvl/internvl_chat
from internvl.train.constants import IMG_CONTEXT_TOKEN

device        = "cuda"
hf_repo       = "OpenGVLab/InternVL3-1B-hf"
finetuned_ckpt= "/content/drive/MyDrive/finetuning-output-Take1"

# 1) Load the original multimodal processor (has the chat template logic)
processor = AutoProcessor.from_pretrained(
    hf_repo,
    trust_remote_code=True,
)

# 2) Load your fine-tuned weights (auto_map points to InternVLChatModel)
model = AutoModelForCausalLM.from_pretrained(
    finetuned_ckpt,
    trust_remote_code=True,
    torch_dtype=torch.float16,
).to(device)

# 3) Tell the model which token marks the start of the visual context
model.img_context_token_id = processor.tokenizer.convert_tokens_to_ids(
    IMG_CONTEXT_TOKEN
)

# 4) Build your message with exactly one video + one text (no stray "<video>")
messages = [
    {
        "role": "user",
        "content": [
            {"type": "video", "url": "/content/drive/MyDrive/PSL_Data_segmentation/Words/Windows.mp4"},
            {"type": "text",  "text": "Which word does the person sign in this video?"},
        ],
    }
]

# 5) Apply the chat template (it’ll load 8 frames, insert <img>…</img> etc.)
inputs = processor.apply_chat_template(
    messages,
    return_tensors="pt",
    add_generation_prompt=False,
    tokenize=True,
    return_dict=True,
    num_frames=16,
).to(device, dtype=torch.float16)

# show exactly what the model sees
prompt_ids = inputs["input_ids"][0]
print(processor.tokenizer.decode(prompt_ids, skip_special_tokens=False))

# 6) Generate and decode
output = model.generate(**inputs, max_new_tokens=25)
decoded_output = processor.tokenizer.decode(output[0], skip_special_tokens=False)
# print(decoded_output)

# Try extracting based on known prompt pattern
if "Answer:" in decoded_output:
    answer = decoded_output.split("Answer:")[-1].strip()
    answer = answer.split("<|im_end|>")[0].strip()
else:
    answer = decoded_output


# # 1a) print out the raw token ID sequence
# print("all token ids:", output[0].tolist())

# # 1b) print the token strings, including special tokens
# print("decoded including specials:",
#       processor.tokenizer.decode(output[0], skip_special_tokens=False))

print("→", answer)


/content/VLM-Finetuning-PSL/internvl/internvl_chat


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


FlashAttention2 is not installed.


Some weights of the model checkpoint at /content/drive/MyDrive/finetuning-output-Take1 were not used when initializing InternVLChatModel: ['language_model.base_model.model.lm_head.weight', 'language_model.base_model.model.model.embed_tokens.weight', 'language_model.base_model.model.model.layers.0.input_layernorm.weight', 'language_model.base_model.model.model.layers.0.mlp.down_proj.base_layer.weight', 'language_model.base_model.model.model.layers.0.mlp.down_proj.lora_A.default.weight', 'language_model.base_model.model.model.layers.0.mlp.down_proj.lora_B.default.weight', 'language_model.base_model.model.model.layers.0.mlp.gate_proj.base_layer.weight', 'language_model.base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'language_model.base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight', 'language_model.base_model.model.model.layers.0.mlp.up_proj.base_layer.weight', 'language_model.base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight', 'lang

<|im_start|>user
Frame1: <img><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CON

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


→ !!!!!!!!!!!!!!!!!!!!!!!!!


Evaluate on folder

In [ ]:
import os
from transformers import AutoProcessor, AutoModelForCausalLM
import torch

%cd /content/VLM-Finetuning-PSL/internvl/internvl_chat
from internvl.train.constants import IMG_CONTEXT_TOKEN

import logging

# Suppress the annoying pad_token_id info from transformers
logging.getLogger("transformers.generation.utils").setLevel(logging.ERROR)

device = "cuda"
hf_repo = "OpenGVLab/InternVL3-1B-hf"
finetuned_ckpt = "/content/drive/MyDrive/finetuning-output-Take1"
video_dir = "/content/drive/MyDrive/PSL_Data_segmentation/Words"

# Load processor and model
processor = AutoProcessor.from_pretrained(
    hf_repo,
    trust_remote_code=True,
)
model = AutoModelForCausalLM.from_pretrained(
    finetuned_ckpt,
    trust_remote_code=True,
    torch_dtype=torch.float16,
).to(device)
model.img_context_token_id = processor.tokenizer.convert_tokens_to_ids(
    IMG_CONTEXT_TOKEN
)

# List all .mp4 videos in the directory
video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]
video_files.sort()

for filename in video_files:
    video_path = os.path.join(video_dir, filename)
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "video", "url": video_path},
                {"type": "text",  "text": "Which word does the person sign in this video?"},
            ],
        }
    ]

    inputs = processor.apply_chat_template(
        messages,
        return_tensors="pt",
        add_generation_prompt=False,
        tokenize=True,
        return_dict=True,
        num_frames=8,
    ).to(device, dtype=torch.float16)

    output = model.generate(**inputs, max_new_tokens=25)
    decoded_output = processor.tokenizer.decode(output[0], skip_special_tokens=False)

    # Extract answer
    if "Answer:" in decoded_output:
        answer = decoded_output.split("Answer:")[-1].strip()
        answer = answer.split("<|im_end|>")[0].strip()
    else:
        answer = decoded_output

    print(f"\n{filename} → {decoded_output}")


/content/VLM-Finetuning-PSL/internvl/internvl_chat

Windows.mp4 → Answer: 'Windows'<|im_end|><|im_end|><|im_end|><|im_end|>10: 00<|im_end|><|im_end|><|im_end|>10: 00<|im_end|>

account.mp4 → <|im_end|>
In this video, the person is seen signing the word 'P' in the PSL (Pakistan Standard Language)

active.mp4 → Answer: PSL<|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|>;<|im_end|><|im_end|>

ambulance.mp4 → <|im_end|>
In this video, the person is seen signing the word 'P' in the Urdu language. The person starts by

bank.mp4 → Answer: 'Kabir'<|im_end|><|im_end|><|im_end|><|im_end|>Question: What is the man's expression?<|im_end|>
Answer: The

blood.mp4 → Answer: 'P'<|im_end|><|im_end|><|im_end|><|im_end|>Question: What is the person doing in the video?<|im_end|>
Answer: The

body.mp4 → Answer: PSL<|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|i

KeyboardInterrupt: 